<a href="https://colab.research.google.com/github/mtrefilek/cs762/blob/main/Feature_Extractor_(ViT).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import torch
from transformers import ViTModel, ViTFeatureExtractor
cwd = os.getcwd().replace('\\','/')

In [ ]:
#%%capture
PRETRAINED_MODEL_NAME = 'vit-base-patch32-384' # 'vit-base-patch16-224'
d_dim = 768

### Model Preparation
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_path = cwd+'/pretrained_models/'+PRETRAINED_MODEL_NAME

# model = ViTModel.from_pretrained('google/'+PRETRAINED_MODEL_NAME)
# processor = ViTFeatureExtractor.from_pretrained('google/'+PRETRAINED_MODEL_NAME)
# model.save_pretrained(model_path)
# processor.save_pretrained(model_path)

model = ViTModel.from_pretrained(model_path)
processor = ViTFeatureExtractor.from_pretrained(model_path)
model.eval()
model.to(device)

Some weights of the model checkpoint at C:/Users/user/Desktop/CS762-Project/pretrained_models/vit-base-patch32-384 were not used when initializing ViTModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing ViTModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ViTModel were not initialized from the model checkpoint at C:/Users/user/Desktop/CS762-Project/pretrained_models/vit-base-patch32-384 and are newly initialized: ['vit.pooler.dense.bias', 'vit.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ViTModel(
  (embeddings): ViTEmbeddings(
    (patch_embeddings): PatchEmbeddings(
      (projection): Conv2d(3, 768, kernel_size=(32, 32), stride=(32, 32))
    )
    (dropout): Dropout(p=0.0, inplace=False)
  )
  (encoder): ViTEncoder(
    (layer): ModuleList(
      (0): ViTLayer(
        (attention): ViTAttention(
          (attention): ViTSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.0, inplace=False)
          )
          (output): ViTSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.0, inplace=False)
          )
        )
        (intermediate): ViTIntermediate(
          (dense): Linear(in_features=768, out_features=3072, bias=True)
        )
        (output): ViTOutput(
          (dense): L

In [ ]:
import numpy as np
import pickle
from copy import deepcopy

In [ ]:
DSET_NAME = 'ChestXRay' #('MNIST', 'FMNIST', 'CIFAR10', 'CIFAR100', 'PlantDisease', 'EuroSAT', 'ChestXRay') 'ISIC2018', 'TinyImageNet' are not yet implemented

In [ ]:
### Dataset Preparation
from torchvision import datasets
from glob import glob, iglob
from PIL import Image

def batch(iterable, n=1):
    l = len(iterable)
    for ndx in range(0, l, n):
        yield iterable[ndx:min(ndx + n, l)]

#import torchvision
#transform=torchvision.transforms.Compose([
    # you can add other transformations in this list
    #torchvision.transforms.ToTensor()
#])
dset_path = cwd + '/dataset'
feature_path = cwd+'/extracted_features/'

if DSET_NAME=='MNIST': # MNIST
    mnist_train = datasets.MNIST(root=dset_path, train=True, download=True)#, transform=transform)
    mnist_test = datasets.MNIST(root=dset_path, train=False, download=True)#, transform=transform)
    imgs_tr, imgs_tst, labels_tr, labels_tst = [], [], [], []
    for (img, label) in mnist_train:
        imgs_tr.append(img.convert(mode='RGB'))
        labels_tr.append(label)
    for (img, label) in mnist_test:
        imgs_tst.append(img.convert(mode='RGB'))
        labels_tst.append(label)
    classnames = [str(a) for a in range(10)]

elif DSET_NAME=='FMNIST': # Fashion-MNIST
    fmnist_train = datasets.FashionMNIST(root=dset_path, train=True, download=True)#, transform=transform)
    fmnist_test = datasets.FashionMNIST(root=dset_path, train=False, download=True)#, transform=transform)
    imgs_tr, imgs_tst, labels_tr, labels_tst = [], [], [], []
    for (img, label) in fmnist_train:
        imgs_tr.append(img.convert(mode='RGB'))
        labels_tr.append(label)
    for (img, label) in fmnist_test:
        imgs_tst.append(img.convert(mode='RGB'))
        labels_tst.append(label)
    classnames = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

elif DSET_NAME=='CIFAR10': # CIFAR-10
    cifar10_train = datasets.CIFAR10(root=dset_path, train=True, download=True)#, transform=transform)
    cifar10_test = datasets.CIFAR10(root=dset_path, train=False, download=True)#, transform=transform)
    imgs_tr, imgs_tst, labels_tr, labels_tst = [], [], [], []
    for (img, label) in cifar10_train:
        imgs_tr.append(img)
        labels_tr.append(label)
    for (img, label) in cifar10_test:
        imgs_tst.append(img)
        labels_tst.append(label)
    meta = pickle.load( open(dset_path+'/cifar-10-batches-py/batches.meta','rb') )
    classnames = meta['label_names']

elif DSET_NAME=='CIFAR100': # CIFAR-100
    cifar100_train = datasets.CIFAR100(root=dset_path, train=True, download=True)#, transform=transform)
    cifar100_test = datasets.CIFAR100(root=dset_path, train=False, download=True)#, transform=transform)
    imgs_tr, imgs_tst, labels_tr, labels_tst = [], [], [], []
    for (img, label) in cifar100_train:
        imgs_tr.append(img)
        labels_tr.append(label)
    for (img, label) in cifar100_test:
        imgs_tst.append(img)
        labels_tst.append(label)
    meta = pickle.load( open(dset_path+'/cifar-100-python/meta','rb') )
    classnames = meta['fine_label_names']

elif DSET_NAME in ('PlantDisease','EuroSAT'): # PlantDisease or EuroSAT
    list_dirs = glob(dset_path+'/'+DSET_NAME+'/*/')
    list_dirs = [l.replace('\\','/') for l in list_dirs]
    classnames = [d.split('/')[-2] for d in list_dirs]
    feature_matrices = []
    for i, classname in enumerate(classnames):
        imgs = []
        for f in iglob(dset_path+'/'+DSET_NAME+'/'+classname+'/*'):
            tmp = Image.open(f)
            img = tmp.copy().convert(mode='RGB')
            imgs.append(img)
        print(classname)
        feature_matrix = np.zeros((1, d_dim))
        for img_batch in batch(imgs, n=32):
            inputs = processor(images=img_batch, return_tensors="pt")
            img_features = model(**inputs.to(device)).pooler_output.cpu().detach().numpy()
            feature_matrix = np.concatenate((feature_matrix, img_features), axis=0)
        feature_matrices.append(deepcopy(feature_matrix[1:]))
        print(classname)
    fname = DSET_NAME+'_'+PRETRAINED_MODEL_NAME+'.npz'
    np.savez(feature_path + fname, feature_matrices = feature_matrices, classnames = classnames)

elif DSET_NAME=='ChestXRay': # Chest X-Ray
    for split in ('train', 'test'):
        list_dirs = glob(dset_path+'/'+DSET_NAME+'/'+split+'/*/')
        list_dirs = [l.replace('\\','/') for l in list_dirs]
        classnames = [d.split('/')[-2] for d in list_dirs]
        feature_matrices = []
        for i, classname in enumerate(classnames):
            feature_matrix = np.zeros((1, d_dim))
            for f in iglob(dset_path+'/'+DSET_NAME+'/'+split+'/'+classname+'/*'):
                tmp = Image.open(f)
                img = tmp.copy().convert(mode='RGB')
                inputs = processor(images=img, return_tensors="pt")
                img_features = model(**inputs.to(device)).pooler_output.cpu().detach().numpy()
                feature_matrix = np.concatenate((feature_matrix, img_features), axis=0)
            feature_matrices.append(deepcopy(feature_matrix[1:]))
            print(classname)
        fname = DSET_NAME+'_'+PRETRAINED_MODEL_NAME+'_'+split+'.npz'
        np.savez(feature_path + fname, feature_matrices = feature_matrices, classnames = classnames)

if DSET_NAME in ('MNIST', 'FMNIST', 'CIFAR10', 'CIFAR100'):
    n_tr = len(imgs_tr)
    feature_matrix_tr = np.zeros((1,d_dim))
    for img_batch in batch(imgs_tr, n=32):
        inputs = processor(images=img_batch, return_tensors="pt")
        img_features_tr = model(**inputs.to(device)).pooler_output.cpu().detach().numpy()
        feature_matrix_tr = np.concatenate((feature_matrix_tr, img_features_tr), axis=0)
        print('Extracting Training Features: {0:.2f}% done'.format(100*len(feature_matrix_tr[1:])/n_tr) )
    feature_matrix_tr = feature_matrix_tr[1:]

    n_cls = np.max(labels_tr)+1
    labels_tr = np.array(labels_tr)
    feature_matrices_tr = []
    for i in range(n_cls):
        feature_matrices_tr.append(feature_matrix_tr[labels_tr==i])

    n_tst = len(imgs_tst)
    feature_matrix_tst = np.zeros((1,d_dim))
    for img_batch in batch(imgs_tst, n=32):
        inputs = processor(images=img_batch, return_tensors="pt")
        img_features_tst = model(**inputs.to(device)).pooler_output.cpu().detach().numpy()
        feature_matrix_tst = np.concatenate((feature_matrix_tst, img_features_tst), axis=0)
        print('Extracting Test Features: {0:.2f}% done'.format(100*len(feature_matrix_tst[1:])/n_tst) )
    feature_matrix_tst = feature_matrix_tst[1:]

    n_cls = np.max(labels_tst)+1
    labels_tst = np.array(labels_tst)
    feature_matrices_tst = []
    for i in range(n_cls):
        feature_matrices_tst.append(feature_matrix_tst[labels_tst==i])
    
### Save Features
if DSET_NAME in ('MNIST', 'FMNIST', 'CIFAR10', 'CIFAR100'):
    fname_tr = DSET_NAME+'_'+PRETRAINED_MODEL_NAME+'_train.npz'
    fname_tst = DSET_NAME+'_'+PRETRAINED_MODEL_NAME+'_test.npz'

    np.savez(feature_path + fname_tr, feature_matrices = feature_matrices_tr, classnames = classnames)
    np.savez(feature_path + fname_tst, feature_matrices = feature_matrices_tst, classnames = classnames)

C:\Users\user\anaconda3\lib\site-packages\transformers\feature_extraction_utils.py:158: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ..\torch\csrc\utils\tensor_new.cpp:201.)
  tensor = as_tensor(value)


NORMAL
PNEUMONIA


C:\Users\user\anaconda3\lib\site-packages\numpy\lib\npyio.py:719: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  val = np.asanyarray(val)


NORMAL
PNEUMONIA
